In [1]:
import numpy as np
import pandas as pd
from utils import *
df = pd.read_csv("SPY.csv")
y = df['Close']

num_states = 2
lags = 2

sig0 = 1
sig1 = 3
p = 0.95
q = 0.95

#Variance prior
d0 = 1 
v0 = 1

#Transition probability prior
u00 = 10 #p11 ~ D(u00,u01)
u01 = 5
u11 = 10 #p11 ~ D(u11,u10)
u10 = 5

#mcmc settings
reps = 5000
burnin = 1000

y = (y-min(y))/(max(y)-min(y)) # Standardize
x = lag_matrix(y, lags, dropna=False) # Get lags
x['constant'] = 1

x = x.iloc[lags:].reset_index(drop=True)
y = y[lags:].reset_index(drop=True)

### Priors ###
b0 = np.zeros((x.shape[1],1)) # num_lags x 1
B0 = np.diag([100]*x.shape[1]) # num_lags x num_lags

a0 = np.zeros((num_states,1)) # num_states x 1
A0 = np.diag([100]*num_states) # num_states x num_states

phi0 = np.zeros((x.shape[1],1))
phi1 = np.zeros((x.shape[1],1))
pmat = np.array([[p,1-p],[1-q,q]])

#storage matrices
out1 = [] #States/Regimes
out2 = [] #Transition Probabilities
out3 = [] #Beta/Variance 

S = np.random.binomial(1, 0.5, size=x.shape[0])
mu = (x.iloc[:,:-1].mean(axis=0)).values


In [2]:
#MCMC

for i in range(1, reps+1):
    
    # Sample probabilities of states per timestep using Hamilton Filter
    S = hamFilt(x=x,
                y=y,
                S1=sig0,
                S2=sig1,
                phi0=phi0,
                phi1=phi1,
                P=pmat,
                S=S,
                mu=mu) 
    # Sample transition probabilities from beta distribution
    pmat = transmat_post(u00 = u00,
                         u01 = u01,
                         u11 = u11,
                         u10 = u10, 
                         S   = S,
                         G   = np.array([1,2]).reshape(2))
    # Sample means (mu), variances (sigma), and coefficients (phi)
    new_params = beta_post(x=x, 
                           y=y, 
                           S=S, 
                           sig0=sig0, 
                           sig1=sig1, 
                           A0=A0, 
                           a0=a0, 
                           B0=B0, 
                           b0=b0, 
                           v0=v0, 
                           d0=d0, 
                           mu=mu, 
                           phi0=phi0.reshape(-1),
                           phi1=phi1.reshape(-1))
    phi0 = new_params['phi0'].reshape(-1,1)
    phi1 = new_params['phi1'].reshape(-1,1)
    sig0 = new_params['sigma0']
    sig1 = new_params['sigma1']
    mu   = new_params['mu']

    # Impose identification restriction on sigma
    if sig0 > sig1:
        S = 1-S
        pmat = pmat[::-1][:,::-1]
        phi0, phi1 = phi1, phi0
        sig0, sig1 = sig1, sig0
        mu = mu[::-1]
        new_params = {'phi0':phi0, 'phi1':phi1, 'sigma0':sig0, 'sigma1':sig1, 'mu':mu}

    out1.append(S)
    out2.append(pmat)
    out3.append(new_params)

    if i>50:
        out1 = out1[-50:]
        out2 = out2[-50:]
        out3 = out3[-50:]


/Users/lorenzoflores/Documents/GitHub/bayes-for-markov-switching/utils.py:39: RuntimeWarning: invalid value encountered in true_divide
  ett11 = ett11/fit
/Users/lorenzoflores/Documents/GitHub/bayes-for-markov-switching/utils.py:41: RuntimeWarning: divide by zero encountered in log
  total_likelihood += np.log(fit)


In [ ]:
#discard burnin
out1 = out1[-c(1:burnin)]
out2 = out2[-c(1:burnin)]
out3 = out3[-c(1:burnin)]

##############compute posterior means
##############compute posterior means
##############compute posterior means
bmr = reps - burnin

s1 = unlist(out1[1])
for i in range(1,bmr):
    s1 = s1 + unlist(out1[i])


postProbs = s1/(bmr)
plot.ts(postProbs)
s1[s1>(bmr)/2] = 1
s1[s1!=1] = 0

plot(y=time.week,x=1:length(time.week),type="l",col="white")
red = which(s1==1)
abline(col="red",v=red)
lines(y=time.week,x=1:length(time.week))

#compute posterior means

betaR0 = matrix(data=NA,ncol=lags+1,nrow = bmr)
betaR1 = matrix(data=NA,ncol=lags+1,nrow = bmr)
sigR0 = c()
sigR1 = c()

for i in range(bmr):
betaR0[i,] = out3[[i]]$phi1
betaR1[i,] = out3[[i]]$phi2
sigR0[i] = out3[[i]]$sig1
sigR1[i] = out3[[i]]$sig2


############## Bayesian Model Averaging for fit
############## Bayesian Model Averaging for fit
############## Bayesian Model Averaging for fit
############## Bayesian Model Averaging for fit
############## Bayesian Model Averaging for fit
############## Bayesian Model Averaging for fit


bR0=colMeans(betaR0)
bR1=colMeans(betaR1)
msigR0=mean(sigR0)
msigR1=mean(sigR1)

credr0 = apply(betaR0,2,FUN=quantile,probs=c(0.025,0.975))
credr1 = apply(betaR1,2,FUN=quantile,probs=c(0.025,0.975))

condMeanR0 = x %*% bR0
condMeanR1 = x %*% bR1

############# Compute posterior transmat
############# Compute posterior transmat
############# Compute posterior transmat
############# Compute posterior transmat
############# Compute posterior transmat


postTransmat = out2[[1]]

for i in range(1,bmr):
postTransmat = postTransmat + out2[[i]]


postTransmat = postTransmat/(bmr)
print(postTransmat)  

############# Compute MSE
############# Compute MSE
############# Compute MSE
############# Compute MSE


MSE_r0 = abs(y - condMeanR0)
MSE_r1 = abs(y - condMeanR1)

MSE_r0 = MSE_r0[which(s1==0)]
MSE_r1 = MSE_r1[which(s1==1)]

mean(MSE_r0)
mean(MSE_r1)

mean(c(MSE_r0,MSE_r1))

save.image("~/nBox/regime_switch_hamfil/sensitivity analysis/output_11Sep_2lag.RData")

#construct output table

tab = cbind(bR0,t(credr0),bR1,t(credr1))
tab = apply(tab,FUN=round,MARGIN=2,3)
tab = tab[-1,]
write.csv(tab,"~/nBox/regime_switch_hamfil/tables/BSR2_output_11Sep.csv")
